In [8]:
from google.colab import userdata
import os
import sys

print("🔐 Loading secrets...")

# -----------------------------
# Load secrets safely
# -----------------------------
JWT_SECRET = userdata.get("JWT_SECRET_KEY")
NGROK_TOKEN = userdata.get("NGROK_AUTHTOKEN")
EMAIL_ID = userdata.get("EMAIL_ID")
EMAIL_PASSWORD = userdata.get("EMAIL_APP_PASSWORD")
ADMIN_EMAIL = userdata.get("ADMIN_EMAIL_ID")
ADMIN_PASSWORD = userdata.get("ADMIN_PASSWORD")

# -----------------------------
# Validate required secrets
# -----------------------------
required_secrets = {
    "JWT_SECRET_KEY": JWT_SECRET,
    "NGROK_AUTHTOKEN": NGROK_TOKEN,
    "EMAIL_ID": EMAIL_ID,
    "EMAIL_APP_PASSWORD": EMAIL_PASSWORD,
    "ADMIN_EMAIL_ID": ADMIN_EMAIL,
    "ADMIN_PASSWORD": ADMIN_PASSWORD
}

missing = [key for key, value in required_secrets.items() if not value]

if missing:
    print("❌ Missing secrets:", ", ".join(missing))
    print("⚠ Please add missing secrets in Colab > Secrets before proceeding.")
    sys.exit(1)
else:
    print("✅ All secrets loaded successfully!")

# -----------------------------
# Set environment variables safely
# -----------------------------
os.environ["JWT_SECRET_KEY"] = JWT_SECRET
os.environ["ACCESS_TOKEN_EXPIRE_MINUTES"] = "30"
os.environ["EMAIL_ID"] = EMAIL_ID
os.environ["EMAIL_APP_PASSWORD"] = EMAIL_PASSWORD
os.environ["ADMIN_EMAIL_ID"] = ADMIN_EMAIL
os.environ["ADMIN_PASSWORD"] = ADMIN_PASSWORD
os.environ["NGROK_AUTHTOKEN"] = NGROK_TOKEN

print("🌍 Environment variables configured.")

# -----------------------------
# Install required packages
# -----------------------------
print("📦 Installing required packages...")

!pip install -q streamlit pyngrok pyjwt watchdog bcrypt
!pip install -q textstat plotly PyPDF2 streamlit-option-menu

print("✅ Setup complete! Ready to proceed.")

🔐 Loading secrets...
✅ All secrets loaded successfully!
🌍 Environment variables configured.
📦 Installing required packages...
✅ Setup complete! Ready to proceed.


In [9]:
%%writefile readability.py
import textstat

class ReadabilityAnalyzer:
    def __init__(self, text):
        # Safe handling for empty or None input
        self.text = text.strip() if text else ""

        # Basic statistics (safe defaults)
        self.num_sentences = textstat.sentence_count(self.text) if self.text else 0
        self.num_words = textstat.lexicon_count(self.text, removepunct=True) if self.text else 0
        self.num_syllables = textstat.syllable_count(self.text) if self.text else 0
        self.complex_words = textstat.difficult_words(self.text) if self.text else 0
        self.char_count = textstat.char_count(self.text) if self.text else 0

    def get_all_metrics(self):
        if not self.text:
            return {
                "Flesch Reading Ease": 0,
                "Flesch-Kincaid Grade": 0,
                "SMOG Index": 0,
                "Gunning Fog": 0,
                "Coleman-Liau": 0
            }

        return {
            "Flesch Reading Ease": textstat.flesch_reading_ease(self.text),
            "Flesch-Kincaid Grade": textstat.flesch_kincaid_grade(self.text),
            "SMOG Index": textstat.smog_index(self.text),
            "Gunning Fog": textstat.gunning_fog(self.text),
            "Coleman-Liau": textstat.coleman_liau_index(self.text)
        }

Overwriting readability.py


In [10]:
%%writefile db.py
import sqlite3
import bcrypt
import datetime
import time

DB_NAME = "users.db"

# ------------------------
# Initialize Database
# ------------------------
def init_db():
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()

        c.execute('''
            CREATE TABLE IF NOT EXISTS users (
                email TEXT PRIMARY KEY,
                password BLOB NOT NULL,
                security_question TEXT NOT NULL,
                security_answer BLOB NOT NULL,
                created_at TEXT NOT NULL
            )
        ''')

        c.execute('''
            CREATE TABLE IF NOT EXISTS password_history (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                email TEXT,
                password BLOB,
                set_at TEXT,
                FOREIGN KEY(email) REFERENCES users(email)
            )
        ''')

        c.execute('''
            CREATE TABLE IF NOT EXISTS login_attempts (
                email TEXT PRIMARY KEY,
                attempts INTEGER DEFAULT 0,
                last_attempt REAL
            )
        ''')

        conn.commit()


# ------------------------
# Register User
# ------------------------
def register_user(email, password, question, answer):
    try:
        with sqlite3.connect(DB_NAME) as conn:
            c = conn.cursor()

            hashed_pw = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
            hashed_answer = bcrypt.hashpw(answer.lower().encode(), bcrypt.gensalt())

            now = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

            c.execute(
                "INSERT INTO users VALUES (?, ?, ?, ?, ?)",
                (email, hashed_pw, question, hashed_answer, now)
            )

            c.execute(
                "INSERT INTO password_history (email, password, set_at) VALUES (?, ?, ?)",
                (email, hashed_pw, now)
            )

            conn.commit()
            return True

    except sqlite3.IntegrityError:
        return False


# ------------------------
# Authenticate User
# ------------------------
def authenticate_user(email, password):
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()
        c.execute("SELECT password FROM users WHERE email = ?", (email,))
        result = c.fetchone()

        if result and bcrypt.checkpw(password.encode(), result[0]):
            _reset_attempts(email)
            return True

    _record_failed_attempt(email)
    return False


# ------------------------
# Check If User Exists
# ------------------------
def check_user_exists(email):
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()
        c.execute("SELECT 1 FROM users WHERE email = ?", (email,))
        return c.fetchone() is not None


# ------------------------
# Update Password
# ------------------------
def update_password(email, new_password):
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()

        hashed_pw = bcrypt.hashpw(new_password.encode(), bcrypt.gensalt())
        now = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

        c.execute("UPDATE users SET password = ? WHERE email = ?", (hashed_pw, email))
        c.execute(
            "INSERT INTO password_history (email, password, set_at) VALUES (?, ?, ?)",
            (email, hashed_pw, now)
        )

        conn.commit()


# ------------------------
# Password History Checks
# ------------------------
def check_is_old_password(email, password):
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()
        c.execute("SELECT password, set_at FROM password_history WHERE email = ?", (email,))
        history = c.fetchall()

    for stored_hash, set_at in history:
        if bcrypt.checkpw(password.encode(), stored_hash):
            return set_at
    return None


def check_password_reused(email, new_password):
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()
        c.execute("SELECT password FROM password_history WHERE email = ?", (email,))
        history = c.fetchall()

    for (stored_hash,) in history:
        if bcrypt.checkpw(new_password.encode(), stored_hash):
            return True
    return False


# ------------------------
# Verify Security Answer
# ------------------------
def verify_security_answer(email, answer):
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()
        c.execute("SELECT security_answer FROM users WHERE email = ?", (email,))
        result = c.fetchone()

    if result:
        return bcrypt.checkpw(answer.lower().encode(), result[0])

    return False


# ------------------------
# Database-Based Rate Limiting
# ------------------------
MAX_ATTEMPTS = 3
LOCKOUT_SECONDS = 60


def _record_failed_attempt(email):
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()
        now = time.time()

        c.execute("SELECT attempts, last_attempt FROM login_attempts WHERE email = ?", (email,))
        row = c.fetchone()

        if row:
            attempts, last = row
            if now - last > LOCKOUT_SECONDS:
                attempts = 1
            else:
                attempts += 1

            c.execute(
                "UPDATE login_attempts SET attempts = ?, last_attempt = ? WHERE email = ?",
                (attempts, now, email)
            )
        else:
            c.execute(
                "INSERT INTO login_attempts VALUES (?, ?, ?)",
                (email, 1, now)
            )

        conn.commit()


def _reset_attempts(email):
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()
        c.execute("DELETE FROM login_attempts WHERE email = ?", (email,))
        conn.commit()


def is_rate_limited(email):
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()
        c.execute("SELECT attempts, last_attempt FROM login_attempts WHERE email = ?", (email,))
        row = c.fetchone()

    if row:
        attempts, last = row
        elapsed = time.time() - last
        if attempts >= MAX_ATTEMPTS and elapsed < LOCKOUT_SECONDS:
            return True, int(LOCKOUT_SECONDS - elapsed)

    return False, 0


# ------------------------
# Admin Functions
# ------------------------
def get_all_users():
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()
        c.execute("SELECT email, created_at FROM users ORDER BY created_at DESC")
        return c.fetchall()


def delete_user(email):
    with sqlite3.connect(DB_NAME) as conn:
        c = conn.cursor()

        c.execute("DELETE FROM password_history WHERE email = ?", (email,))
        c.execute("DELETE FROM login_attempts WHERE email = ?", (email,))
        c.execute("DELETE FROM users WHERE email = ?", (email,))

        conn.commit()

Overwriting db.py


In [11]:

import os
import db

# Initialize database
db.init_db()

# Fetch admin credentials safely from environment
ADMIN_EMAIL = os.getenv("ADMIN_EMAIL_ID")
ADMIN_PASSWORD = os.getenv("ADMIN_PASSWORD")

if not ADMIN_EMAIL or not ADMIN_PASSWORD:
    print("❌ Admin credentials missing in environment variables.")
else:
    if not db.check_user_exists(ADMIN_EMAIL):
        created = db.register_user(
            ADMIN_EMAIL,
            ADMIN_PASSWORD,
            "Admin Default Question",
            "admin123"
        )

        if created:
            print("✅ Admin account created successfully.")
        else:
            print("⚠ Failed to create admin account.")
    else:
        print("ℹ Admin already exists.")

ℹ Admin already exists.


In [12]:
import os
from pyngrok import ngrok

print("⚙ Configuring Streamlit environment...")

# -----------------------------
# Validate NGROK token
# -----------------------------
NGROK_TOKEN = os.getenv("NGROK_AUTHTOKEN")

if not NGROK_TOKEN:
    raise ValueError("NGROK_AUTHTOKEN not found in environment variables.")

ngrok.set_auth_token(NGROK_TOKEN)

# -----------------------------
# Create .streamlit folder
# -----------------------------
os.makedirs(".streamlit", exist_ok=True)

# -----------------------------
# Streamlit Dark Theme Config
# -----------------------------
config_toml = """
[theme]
base = "dark"
primaryColor = "#4F8BF9"
backgroundColor = "#0E1117"
secondaryBackgroundColor = "#262730"
textColor = "#FAFAFA"
font = "sans serif"

[server]
headless = true
enableCORS = false
enableXsrfProtection = false
"""

config_path = ".streamlit/config.toml"

# Write config safely
with open(config_path, "w") as f:
    f.write(config_toml)

print("🎨 Dark Theme configuration applied successfully.")

⚙ Configuring Streamlit environment...
🎨 Dark Theme configuration applied successfully.


In [13]:
%%writefile app.py

import streamlit as st
import jwt
import datetime
import time
import re
import db
import bcrypt
import os
import random
import sqlite3

# continue your full streamlit code here...


import readability
import plotly.graph_objects as go
import PyPDF2
from streamlit_option_menu import option_menu



# --- Styling ---
st.set_page_config(page_title="Policy Nav Login", page_icon="🤖", layout="centered")




#otp generation

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import secrets
import hmac
import hashlib
import struct

OTP_EXPIRY_MINUTES = 10

def generate_secure_otp():
    secret = secrets.token_bytes(20)
    counter = int(time.time())
    msg = struct.pack(">Q", counter)

    h = hmac.new(secret, msg, hashlib.sha1).digest()
    offset = h[19] & 0xf

    code = ((h[offset] & 0x7f) << 24 |
            (h[offset + 1] & 0xff) << 16 |
            (h[offset + 2] & 0xff) << 8 |
            (h[offset + 3] & 0xff))

    return str(code % 1000000).zfill(6)

# --- Email Logic ---
def send_email(to_email, otp, app_pass=None):
    msg = MIMEMultipart()
    msg['From'] = f"Policy Nav <{EMAIL_ADDRESS}>"
    msg['To'] = to_email
    msg['Subject'] = "🔐 Policy Nav- Password Reset OTP"

    body = f"""
    <!DOCTYPE html>
    <html>
    <head>
    <style>
    .container {{
        font-family: 'Courier New', monospace;
        background-color: #0e1117;
        padding: 40px;
        text-align: center;
        color: #ffffff;
    }}
    .card {{
        background-color: #1f2937;
        border-radius: 12px;
        box-shadow: 0 0 20px rgba(0, 255, 204, 0.2);
        padding: 40px;
        max-width: 500px;
        margin: 0 auto;
        border: 1px solid #374151;
    }}
    .header {{
        color: #00ffcc;
        font-size: 24px;
        font-weight: 600;
        margin-bottom: 20px;
        text-shadow: 0 0 5px #00ffcc;
    }}
    .otp-box {{
        background-color: #0e1117;
        color: #00ffcc;
        font-size: 32px;
        font-weight: 700;
        letter-spacing: 8px;
        padding: 20px;
        border-radius: 8px;
        margin: 30px 0;
        display: inline-block;
        border: 1px solid #00ffcc;
        box-shadow: 0 0 10px rgba(0, 255, 204, 0.3);
    }}
    .text {{
        color: #9ca3af;
        font-size: 16px;
        line-height: 1.5;
        margin-bottom: 20px;
    }}
    .footer {{
        color: #6b7280;
        font-size: 12px;
        margin-top: 30px;
    }}
    </style>
    </head>
    <body>
    <div class="container">
        <div class="card">
            <div class="header">⚡ Policy Nav Security</div>
            <div class="text">
                Use this OTP to reset your password for
                <span style="color:#00ffcc;">{to_email}</span>.
            </div>
            <div class="otp-box">{otp}</div>
            <div class="text">
                Valid for <strong>{OTP_EXPIRY_MINUTES} minutes</strong>.
            </div>
            <div class="footer">
                &copy; 2026 Policy Nav Secure Auth
            </div>
        </div>
    </div>
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()

        password_to_use = app_pass if app_pass else EMAIL_PASSWORD

        if not password_to_use:
            return False, "No App Password found. Check Secrets."

        server.login(EMAIL_ADDRESS, password_to_use)
        server.sendmail(EMAIL_ADDRESS, to_email, msg.as_string())
        server.quit()

        return True, "Email sent successfully!"

    except Exception as e:
        return False, str(e)


# Initialize Database
if 'db_initialized' not in st.session_state:
    db.init_db()
    st.session_state['db_initialized'] = True


# --- Configuration ---
SECRET_KEY = os.getenv("JWT_SECRET_KEY")
if not SECRET_KEY:
    st.error("JWT_SECRET_KEY is not configured.")
    st.stop()

ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = int(os.getenv("ACCESS_TOKEN_EXPIRE_MINUTES", 30))


# --- Email Configuration ---
EMAIL_ADDRESS = os.getenv("EMAIL_ID")
EMAIL_PASSWORD = os.getenv("EMAIL_APP_PASSWORD")

if not EMAIL_ADDRESS:
    st.error("EMAIL_ID not configured.")
    st.stop()


# --- JWT Utils ---
def create_access_token(data: dict):
    to_encode = data.copy()
    expire = datetime.datetime.utcnow() + datetime.timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt
def verify_token(token: str):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except jwt.ExpiredSignatureError:
        return None
    except jwt.InvalidTokenError:
        return None
# --- Validation Utils ---
def is_valid_email(email):
    # Regex for standard email format
    email= email.strip()
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.fullmatch(pattern, email) is not None

def check_password_strength(password):
    has_upper = bool(re.search(r"[A-Z]", password))
    has_lower = bool(re.search(r"[a-z]", password))
    has_digit = bool(re.search(r"\d", password))
    has_special = bool(re.search(r'[!@#$%^&*(),.?":{}|<>]', password))

    if len(password) >= 8 and has_upper and has_lower and has_digit and has_special:
        return "Strong"
    elif len(password) >= 6:
        return "Medium"
    else:
        return "Weak"
def create_otp_token(otp, email):
    otp_hash = bcrypt.hashpw(otp.encode(), bcrypt.gensalt()).decode()

    payload = {
        "otp_hash": otp_hash,
        "sub": email,
        "type": "otp_verification",
        "exp": datetime.datetime.utcnow() + datetime.timedelta(minutes=OTP_EXPIRY_MINUTES)
    }

    return jwt.encode(payload, SECRET_KEY, algorithm=ALGORITHM)
def verify_otp_token(token, input_otp, email):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])

        if payload.get("type") != "otp_verification":
            return False

        if payload.get("sub") != email:
            return False

        if bcrypt.checkpw(input_otp.encode(), payload["otp_hash"].encode()):
            return True

        return False

    except Exception:
        return False


MAX_OTP_ATTEMPTS = 3

def record_otp_attempt():
    st.session_state['otp_attempts'] = st.session_state.get('otp_attempts', 0) + 1



def is_valid_password(password):
    # Alphanumeric check and min length 8
    if len(password) < 8:
        return False
    if not password.isalnum():
        return False
    return True
# --- Session State Management ---
if 'jwt_token' not in st.session_state:
    st.session_state['jwt_token'] = None
if 'page' not in st.session_state:
    st.session_state['page'] = 'login'
# Mock Database (In-memory for demo)
# Structure: {email: {'password': password, 'username': username, ...}}
# Also store usernames separately for quick check: {username: email}




st.markdown("""
<style>

/* =====================================================
   GLOBAL RESET
===================================================== */

html, body {
    margin: 0;
    padding: 0;
    font-family: 'Segoe UI', sans-serif;
    background: radial-gradient(circle at 30% 20%, #0f2027, #203a43, #0e1117);
    color: white;
    overflow-x: hidden;
}

/* Proper Scroll Handling */
[data-testid="stAppViewContainer"] {
    height: 100vh;
    overflow-y: auto;
    padding: 0 !important;
    margin: 0 !important;
}

.main {
    padding-top: 0 !important;
    margin: 0 !important;
}

/* =====================================================
   REMOVE STREAMLIT HEADER & DEFAULT SPACING
===================================================== */

header,
[data-testid="stToolbar"],
[data-testid="stDecoration"],
[data-testid="stStatusWidget"] {
    display: none !important;
}

.block-container {
    padding: 0 !important;
    margin: 0 auto !important;
    max-width: 100% !important;
}

/* =====================================================
   AUTH FULLSCREEN CENTER
===================================================== */


/* =====================================================
   GLASS AUTH CARD
===================================================== */



    backdrop-filter: blur(25px);
    -webkit-backdrop-filter: blur(25px);

    border: 1px solid rgba(0,255,255,0.25);

    box-shadow:
        0 0 40px rgba(0,255,255,0.25),
        0 12px 40px rgba(0,0,0,0.6);

    transition: all 0.3s ease;
}

.auth-wrapper {
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 100vh;   /* Perfect vertical center */
}


.auth-card {
    width: 400px;
    max-width: 95%;
    padding: 2rem;
    border-radius: 12px;

    background: #111827;        /* solid background */
    border: 1px solid #374151;  /* simple border */

    box-shadow: none;           /* remove glow */
    backdrop-filter: none;      /* remove glass blur */
}

/* =====================================================
   TITLES
===================================================== */

.auth-title {
    font-size: 28px;
    font-weight: 700;
    text-align: center;
    margin-bottom: 6px;
}

.auth-sub {
    text-align: center;
    color: #cbd5e1;
    margin-bottom: 25px;
    font-size: 14px;
}

/* =====================================================
   INPUT FIELDS
===================================================== */

div[data-baseweb="input"] input,
div[data-baseweb="textarea"] textarea {
    background: rgba(255,255,255,0.08) !important;
    border: 1px solid rgba(0,255,255,0.35) !important;
    border-radius: 16px !important;
    padding: 14px !important;
    color: white !important;
    transition: all 0.25s ease !important;
}

div[data-baseweb="input"] input:focus,
div[data-baseweb="textarea"] textarea:focus {
    border: 1px solid #00ffff !important;
    box-shadow: 0 0 12px #00ffff !important;
    outline: none !important;
}

input::placeholder,
textarea::placeholder {
    color: #9ca3af !important;
}

/* Select Styling */
div[data-baseweb="select"] {
    background: rgba(255,255,255,0.08) !important;
    border-radius: 16px !important;
    border: 1px solid rgba(0,255,255,0.35) !important;
}

div[data-baseweb="popover"] {
    background-color: #1f2937 !important;
}

/* =====================================================
   BUTTONS
===================================================== */

div.stButton > button,
button[data-testid="baseButton-primary"] {
    background: linear-gradient(135deg, #00c6ff, #0072ff) !important;
    border-radius: 18px !important;
    border: none !important;
    padding: 14px !important;
    font-weight: 600 !important;
    color: white !important;
    transition: all 0.25s ease !important;
    width: 100%;
}

div.stButton > button:hover,
button[data-testid="baseButton-primary"]:hover {
    box-shadow: 0 0 20px #00ffff !important;
    transform: translateY(-2px);
}

/* =====================================================
   CHECKBOX / RADIO
===================================================== */

.stCheckbox label,
.stRadio label {
    color: #cbd5e1 !important;
}

/* =====================================================
   ALERTS
===================================================== */

.stAlert {
    border-radius: 12px !important;
}

/* =====================================================
   SIDEBAR
===================================================== */

section[data-testid="stSidebar"] {
    background: rgba(15,23,42,0.95);
    border-right: 1px solid rgba(0,255,255,0.25);
}

section[data-testid="stSidebar"] button {
    border-radius: 12px !important;
}

/* =====================================================
   DASHBOARD CARDS
===================================================== */

.readability-card {
    background: #1f2937;
    padding: 25px;
    border-radius: 14px;
    border: 1px solid #374151;
    box-shadow: 0 0 20px rgba(0,255,255,0.15);
}

.overall-box {
    background: #111827;
    padding: 20px;
    border-radius: 12px;
    border-left: 6px solid #00ffff;
    box-shadow: 0 0 15px rgba(0,255,255,0.3);
}

.metric-box {
    background: #111827;
    padding: 15px;
    border-radius: 10px;
    border: 1px solid #374151;
    text-align: center;
}

.section-divider {
    height: 2px;
    background: linear-gradient(to right, #00ffff, transparent);
    margin: 25px 0;
}

.gauge-card {
    background: rgba(15,23,42,0.7);
    padding: 20px;
    border-radius: 18px;
    border: 1px solid rgba(0,255,255,0.25);
    box-shadow: 0 0 20px rgba(0,255,255,0.25);
    transition: 0.3s ease;
}

.gauge-card:hover {
    box-shadow: 0 0 35px rgba(0,255,255,0.45);
    transform: translateY(-4px);
}

/* =====================================================
   CHAT UI
===================================================== */

.chat-box {
    height: 400px;
    overflow-y: auto;
    padding: 15px;
    border-radius: 16px;
    background: rgba(255,255,255,0.05);
    margin-bottom: 20px;
}

.user-msg {
    background: linear-gradient(135deg, #00c6ff, #0072ff);
    padding: 10px 14px;
    border-radius: 14px;
    margin: 8px 0;
    text-align: right;
}

.bot-msg {
    background: rgba(255,255,255,0.08);
    padding: 10px 14px;
    border-radius: 14px;
    margin: 8px 0;
    text-align: left;
}

/* =====================================================
   CUSTOM SCROLLBAR
===================================================== */

::-webkit-scrollbar {
    width: 6px;
}

::-webkit-scrollbar-thumb {
    background: #00ffff;
    border-radius: 10px;
}

/* =====================================================
   MOBILE RESPONSIVENESS
===================================================== */

@media (max-width: 768px) {
    .auth-card {
        width: 95%;
        padding: 1.5rem;
    }

    .chat-box {
        height: 300px;
    }
}

</style>
""", unsafe_allow_html=True)

def hide_sidebar_for_auth():
    st.markdown("""
        <style>
            [data-testid="stSidebar"] {display: none;}
        </style>
    """, unsafe_allow_html=True)


def glass_card_start():
    st.markdown('<div class="glass-box">', unsafe_allow_html=True)

def glass_card_end():
    st.markdown('</div>', unsafe_allow_html=True)


def create_gauge(value, title, min_val=0, max_val=100):

    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=value,
        title={
            'text': f"<b>{title}</b>",
            'font': {'size': 18, 'color': "#ffffff"}
        },
        number={'font': {'size': 28, 'color': "#60a5fa"}},
        gauge={
            'axis': {
                'range': [min_val, max_val],
                'tickcolor': "#9ca3af"
            },
            'bar': {'color': "#3b82f6"},
            'bgcolor': "rgba(0,0,0,0)",
            'borderwidth': 0,
            'steps': [
                {'range': [min_val, max_val], 'color': "rgba(59,130,246,0.15)"}
            ]
        }
    ))

    fig.update_layout(
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=20, r=20, t=40, b=20),
        height=250
    )

    return fig

# --- Views ---
def login_page():

    hide_sidebar_for_auth()

    # ===== WRAPPER START =====
    st.markdown('<div class="auth-wrapper">', unsafe_allow_html=True)
    st.markdown('<div class="auth-card">', unsafe_allow_html=True)

    st.markdown("<div class='auth-title'>Welcome Back</div>", unsafe_allow_html=True)
    st.markdown("<div class='auth-sub'>Sign in to your account</div>", unsafe_allow_html=True)

    # ---------- LOGIN FORM ----------
    with st.form("login_form"):

        email = st.text_input("Email Address").strip()

        show_password = st.checkbox("Show Password")

        password = st.text_input(
            "Password",
            type="text" if show_password else "password"
        )

        col1, col2 = st.columns(2)

        with col1:
            remember = st.checkbox("Remember me")

        with col2:
            forgot_clicked = st.form_submit_button("Forgot Password?")

        login_clicked = st.form_submit_button("Sign In")

        # ---------- FORGOT ----------
        if forgot_clicked:
            st.session_state['page'] = 'forgot'
            st.rerun()

        # ---------- LOGIN ----------
        if login_clicked:

            if not email or not password:
                st.error("Please enter both email and password.")
            else:

                is_locked, wait_time = db.is_rate_limited(email)

                if is_locked:
                    st.error(f"Account locked. Try again in {int(wait_time)} seconds.")
                else:
                    with st.spinner("Authenticating..."):
                        time.sleep(1)

                        if db.authenticate_user(email, password):

                            db._reset_attempts(email)

                            token = create_access_token({"sub": email})
                            st.session_state['jwt_token'] = token

                            # Optional remember logic
                            if remember:
                                st.session_state['remember_user'] = True

                            st.success("Login successful!")
                            time.sleep(1)
                            st.rerun()

                        else:
                            st.error("Invalid credentials.")

    st.markdown("<hr style='margin:20px 0;'>", unsafe_allow_html=True)

    # ---------- CREATE ACCOUNT ----------
    if st.button("Create an Account", use_container_width=True):
        st.session_state['page'] = 'signup'
        st.rerun()

    # ===== WRAPPER END =====
    st.markdown('</div>', unsafe_allow_html=True)
    st.markdown('</div>', unsafe_allow_html=True)

def signup_page():
    hide_sidebar_for_auth()

    # ===== WRAPPER START (IMPORTANT) =====
    st.markdown('<div class="auth-wrapper">', unsafe_allow_html=True)
    st.markdown('<div class="auth-card">', unsafe_allow_html=True)

    st.markdown("<div class='auth-title'>Create Account</div>", unsafe_allow_html=True)
    st.markdown("<div class='auth-sub'>Join Policy Nav securely</div>", unsafe_allow_html=True)

    # ---------------- USER DETAILS ----------------
    email = st.text_input("Email Address").strip()
    password = st.text_input("Password", type="password")
    confirm_password = st.text_input("Confirm Password", type="password")

    security_question = st.selectbox(
        "Select Security Question",
        [
            "What is your childhood school name?",
            "What is your favorite color?",
            "What is your pet's name?",
            "What city were you born in?"
        ]
    )

    security_answer = st.text_input("Security Answer")

    # ---------------- PASSWORD STRENGTH ----------------
    if password:
        strength = check_password_strength(password)

        color_map = {
            "Weak": "red",
            "Medium": "orange",
            "Strong": "lime"
        }

        st.markdown(
            f"<div style='color:{color_map[strength]};font-weight:bold;'>Strength: {strength}</div>",
            unsafe_allow_html=True
        )

    # ---------------- SEND OTP ----------------
    if st.button("Send OTP", use_container_width=True):

        error_flag = False

        if not email or not password or not confirm_password:
            st.error("All fields are required.")
            error_flag = True

        elif not security_answer.strip():
            st.error("Security answer is required.")
            error_flag = True

        elif not is_valid_email(email):
            st.error("Invalid email format.")
            error_flag = True

        elif password != confirm_password:
            st.error("Passwords do not match.")
            error_flag = True

        elif not is_valid_password(password):
            st.error("Password must be at least 8 characters and alphanumeric.")
            error_flag = True

        elif db.check_user_exists(email):
            st.error("User already exists.")
            error_flag = True

        elif db.check_password_reused(email, password):
            st.error("Old password reuse is not allowed.")
            error_flag = True

        if not error_flag:
            st.session_state['temp_signup'] = {
                "email": email,
                "password": password,
                "security_question": security_question,
                "security_answer": security_answer
            }

            otp = generate_secure_otp()
            token = create_otp_token(otp, email)

            st.session_state['otp_token'] = token
            st.session_state['otp_attempts'] = 0
            st.session_state['otp_last_sent'] = time.time()

            success, message = send_email(email, otp)

            if success:
                st.success(message)
            else:
                st.error(f"Email failed: {message}")

    # ---------------- OTP VERIFICATION ----------------
    if 'otp_token' in st.session_state and 'temp_signup' in st.session_state:

        user_otp = st.text_input("Enter OTP")

        if st.button("Verify OTP and Create Account", use_container_width=True):

            valid = verify_otp_token(
                st.session_state.get("otp_token"),
                user_otp,
                st.session_state['temp_signup']['email']
            )

            if valid:
                data = st.session_state['temp_signup']

                try:
                    db.register_user(
                        data["email"],
                        data["password"],
                        data["security_question"],
                        data["security_answer"]
                    )

                    st.success("Account created successfully!")

                    for key in ['otp_token', 'temp_signup', 'otp_attempts']:
                        st.session_state.pop(key, None)

                    time.sleep(1)
                    st.session_state['page'] = 'login'
                    st.rerun()

                except Exception:
                    st.error("Registration failed.")

            else:
                record_otp_attempt()
                remaining = MAX_OTP_ATTEMPTS - st.session_state.get('otp_attempts', 0)

                if remaining > 0:
                    st.error(f"Invalid OTP. Attempts left: {remaining}")
                else:
                    st.error("Too many failed OTP attempts.")
                    for key in ['otp_token', 'temp_signup', 'otp_attempts']:
                        st.session_state.pop(key, None)

    # ===== CLOSE CARD + WRAPPER =====
    st.markdown('</div>', unsafe_allow_html=True)  # auth-card
    st.markdown('</div>', unsafe_allow_html=True)  # auth-wrapper


def forgot_password_page():

    hide_sidebar_for_auth()

    # ===== WRAPPER START =====
    st.markdown('<div class="auth-wrapper">', unsafe_allow_html=True)
    st.markdown('<div class="auth-card">', unsafe_allow_html=True)

    st.markdown("<div class='auth-title'>Reset Password</div>", unsafe_allow_html=True)
    st.markdown("<div class='auth-sub'>Recover your account securely</div>", unsafe_allow_html=True)

    # ================= EMAIL VERIFICATION =================
    with st.form("verify_email_form"):

        email = st.text_input("Enter your registered Email")
        verify_email_btn = st.form_submit_button("Verify Email")

        if verify_email_btn:

            if not db.check_user_exists(email):
                st.error("Email not found.")
            else:
                st.session_state['reset_email'] = email
                st.success("Choose verification method.")

    # ================= METHOD SELECTION =================
    if 'reset_email' in st.session_state:

        method = st.radio("Choose Verification Method", ["OTP", "Security Question"])

        # ================= OTP METHOD =================
        if method == "OTP":

            if st.button("Send OTP"):

                otp = generate_secure_otp()
                token = create_otp_token(
                    otp,
                    st.session_state['reset_email']
                )

                st.session_state['otp_token'] = token
                st.session_state['otp_attempts'] = 0

                success, message = send_email(
                    st.session_state['reset_email'],
                    otp
                )

                if success:
                    st.success(message)
                else:
                    st.error(f"Email failed: {message}")

            if 'otp_token' in st.session_state:

                with st.form("otp_form"):

                    user_otp = st.text_input("Enter OTP")
                    verify_otp_btn = st.form_submit_button("Verify OTP")

                    if verify_otp_btn:

                        valid = verify_otp_token(
                            st.session_state.get("otp_token"),
                            user_otp,
                            st.session_state['reset_email']
                        )

                        if valid:
                            st.session_state['verified'] = True
                            st.success("OTP verified successfully!")
                            st.session_state.pop('otp_attempts', None)
                        else:
                            record_otp_attempt()
                            remaining = MAX_OTP_ATTEMPTS - st.session_state.get('otp_attempts', 0)

                            if remaining > 0:
                                st.error(f"Invalid OTP. Attempts left: {remaining}")
                            else:
                                st.error("Too many failed OTP attempts.")
                                for key in ['otp_token', 'otp_attempts']:
                                    st.session_state.pop(key, None)

        # ================= SECURITY QUESTION =================
        else:

            conn = sqlite3.connect("users.db")
            c = conn.cursor()
            c.execute(
                "SELECT security_question FROM users WHERE email = ?",
                (st.session_state['reset_email'],)
            )
            result = c.fetchone()
            conn.close()

            if result:

                with st.form("security_form"):

                    st.info(result[0])
                    answer = st.text_input("Enter Security Answer")
                    verify_answer_btn = st.form_submit_button("Verify Answer")

                    if verify_answer_btn:

                        if db.verify_security_answer(
                            st.session_state['reset_email'],
                            answer
                        ):
                            st.session_state['verified'] = True
                            st.success("Answer verified!")
                        else:
                            st.error("Incorrect answer")

        # ================= RESET PASSWORD =================
        if st.session_state.get('verified'):

            with st.form("reset_password_form"):

                new_password = st.text_input("New Password", type="password")
                confirm_password = st.text_input("Confirm Password", type="password")

                if new_password:
                    strength = check_password_strength(new_password)

                    color_map = {
                        "Weak": "red",
                        "Medium": "orange",
                        "Strong": "lime"
                    }

                    st.markdown(
                        f"<div style='color:{color_map[strength]};font-weight:bold;'>Strength: {strength}</div>",
                        unsafe_allow_html=True
                    )

                reset_btn = st.form_submit_button("Reset Password")

                if reset_btn:

                    if new_password != confirm_password:
                        st.error("Passwords do not match.")

                    elif not is_valid_password(new_password):
                        st.error("Invalid password format.")

                    elif db.check_password_reused(
                        st.session_state['reset_email'],
                        new_password
                    ):
                        st.error("Old password reuse is not allowed.")

                    else:
                        db.update_password(
                            st.session_state['reset_email'],
                            new_password
                        )

                        st.success("Password updated successfully!")

                        for key in [
                            'reset_email',
                            'verified',
                            'otp_token',
                            'otp_attempts'
                        ]:
                            st.session_state.pop(key, None)

                        time.sleep(1)
                        st.session_state['page'] = 'login'
                        st.rerun()

    # ================= BACK BUTTON =================
    if st.button("Back to Login", use_container_width=True):
        st.session_state['page'] = 'login'
        st.rerun()

    # ===== WRAPPER END =====
    st.markdown('</div>', unsafe_allow_html=True)
    st.markdown('</div>', unsafe_allow_html=True)




def dashboard_page():
    token = st.session_state.get('jwt_token')
    payload = verify_token(token)

    if not payload:
        st.session_state['jwt_token'] = None
        st.warning("Session expired or invalid. Please login again.")
        time.sleep(1)
        st.rerun()
        return

    email = payload.get("sub")
    username = email.split("@")[0]


    ADMIN_EMAIL = os.getenv("ADMIN_EMAIL_ID")



    # ---------- Initialize Chat ----------
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = [
            {"role": "bot", "content": "Hello! I am LLM. Ask me anything about LLM!"}
        ]



    # ---------- HEADER ----------
    st.markdown(
        f'''
        <div style="text-align:center; margin-top:20px;">
            <h1 style="color:white;">Welcome, {username.upper()}!</h1>
            <h3 style="color:#cbd5e1;">How can I help you today?</h3>
        </div>
        ''',
        unsafe_allow_html=True
    )



    # ---------- CHAT BOX ----------
    chat_html = '<div class="chat-box">'

    for message in st.session_state.chat_history:
        if message["role"] == "user":
            chat_html += f'<div class="user-msg">{message["content"]}</div>'
        else:
            chat_html += f'<div class="bot-msg">{message["content"]}</div>'

    chat_html += '</div>'

    st.markdown(chat_html, unsafe_allow_html=True)



    # ---------- INPUT ----------
    with st.form("chat_form", clear_on_submit=True):
        col1, col2 = st.columns([6, 1])

        with col1:
            user_input = st.text_input("", placeholder="Ask me anything...", label_visibility="collapsed")

        with col2:
            submit = st.form_submit_button("Send")

        if submit and user_input:
            st.session_state.chat_history.append(
                {"role": "user", "content": user_input}
            )

            st.session_state.chat_history.append(
                {"role": "bot", "content": "I am a demo bot. I received your message!"}
            )

            st.rerun()


def readability_page():

    token = st.session_state.get('jwt_token')
    payload = verify_token(token)

    if not payload:
        st.warning("Please login first.")
        st.session_state['page'] = "login"
        st.rerun()
        return

    st.title("📖 Text Readability Analyzer")

    tab1, tab2 = st.tabs(["✍️ Input Text", "📂 Upload File"])

    text_input = ""

    with tab1:
        raw_text = st.text_area("Enter text (min 50 characters):", height=200)
        if raw_text:
            text_input = raw_text

    with tab2:
        uploaded_file = st.file_uploader("Upload TXT or PDF", type=["txt", "pdf"])
        if uploaded_file:
            try:
                if uploaded_file.type == "application/pdf":
                    reader = PyPDF2.PdfReader(uploaded_file)
                    text = ""
                    for page in reader.pages:
                        text += page.extract_text() + "\n"
                    text_input = text
                else:
                    text_input = uploaded_file.read().decode("utf-8")
            except:
                st.error("File reading error.")

    if st.button("Analyze Readability", use_container_width=True):

        if len(text_input) < 50:
            st.error("Text too short (minimum 50 characters).")
            return

        analyzer = readability.ReadabilityAnalyzer(text_input)
        scores = analyzer.get_all_metrics()

        # =========================
        # 🔥 DASHBOARD HEADER
        # =========================
        st.markdown('<div class="section-divider"></div>', unsafe_allow_html=True)

        st.markdown("""
        <div class="readability-card">
            <h2 style="text-align:center; color:#00ffcc;">
                📊 Readability Analysis Dashboard
            </h2>
        </div>
        """, unsafe_allow_html=True)

        # =========================
        # 🔥 OVERALL GRADE LEVEL
        # =========================

        avg_grade = (
            scores["Flesch-Kincaid Grade"]
            + scores["Gunning Fog"]
            + scores["SMOG Index"]
            + scores["Coleman-Liau"]
        ) / 4

        if avg_grade <= 6:
            level = "Beginner (Elementary)"
            color = "#22c55e"
        elif avg_grade <= 10:
            level = "Intermediate (Middle School)"
            color = "#06b6d4"
        elif avg_grade <= 14:
            level = "Advanced (High School / College)"
            color = "#facc15"
        else:
            level = "Expert (Academic / Professional)"
            color = "#ef4444"

        st.markdown(f"""
        <div class="overall-box" style="border-left:6px solid {color};">
            <h2 style="color:{color}; margin:0;">
                Overall Level: {level}
            </h2>
            <p style="color:#9ca3af; margin-top:5px;">
                Approximate Grade Level: {int(avg_grade)}
            </p>
        </div>
        """, unsafe_allow_html=True)

        # =========================
        # 🔥 GAUGE SECTION
        # =========================
        st.markdown('<div class="section-divider"></div>', unsafe_allow_html=True)
        st.subheader("📈 Readability Scores")

        col1, col2, col3 = st.columns(3)

        with col1:
          st.markdown('<div class="gauge-card">', unsafe_allow_html=True)
          st.plotly_chart(
              create_gauge(scores["Flesch Reading Ease"], "Flesch Ease", 0, 100),
              use_container_width=True
          )
        st.markdown('</div>', unsafe_allow_html=True)


        with col2:
            st.markdown('<div class="gauge-card">', unsafe_allow_html=True)
            st.plotly_chart(
                create_gauge(scores["Flesch-Kincaid Grade"], "Kincaid Grade", 0, 20),
                use_container_width=True
            )
            st.markdown('</div>', unsafe_allow_html=True)

        with col3:
            st.markdown('<div class="gauge-card">', unsafe_allow_html=True)
            st.plotly_chart(
                create_gauge(scores["SMOG Index"], "SMOG Index", 0, 20),
                use_container_width=True
            )
            st.markdown('</div>', unsafe_allow_html=True)

        col4, col5 = st.columns(2)

        with col4:
            st.markdown('<div class="gauge-card">', unsafe_allow_html=True)
            st.plotly_chart(
                create_gauge(scores["Gunning Fog"], "Gunning Fog", 0, 20),
                use_container_width=True
            )
            st.markdown('</div>', unsafe_allow_html=True)


        with col5:
            st.markdown('<div class="gauge-card">', unsafe_allow_html=True)
            st.plotly_chart(
                create_gauge(scores["Coleman-Liau"], "Coleman-Liau", 0, 20),
                use_container_width=True
            )
            st.markdown('</div>', unsafe_allow_html=True)

        # =========================
        # 🔥 TEXT STATISTICS
        # =========================

        st.markdown('<div class="section-divider"></div>', unsafe_allow_html=True)
        st.subheader("📝 Text Statistics")

        c1, c2, c3, c4, c5 = st.columns(5)

        c1.markdown(f"""
        <div class="metric-box">
            <h3>{analyzer.num_sentences}</h3>
            <p>Sentences</p>
        </div>
        """, unsafe_allow_html=True)

        c2.markdown(f"""
        <div class="metric-box">
            <h3>{analyzer.num_words}</h3>
            <p>Words</p>
        </div>
        """, unsafe_allow_html=True)

        c3.markdown(f"""
        <div class="metric-box">
            <h3>{analyzer.num_syllables}</h3>
            <p>Syllables</p>
        </div>
        """, unsafe_allow_html=True)

        c4.markdown(f"""
        <div class="metric-box">
            <h3>{analyzer.complex_words}</h3>
            <p>Complex Words</p>
        </div>
        """, unsafe_allow_html=True)

        c5.markdown(f"""
        <div class="metric-box">
            <h3>{analyzer.char_count}</h3>
            <p>Characters</p>
        </div>
        """, unsafe_allow_html=True)




def admin_page():

    # -------- AUTH CHECK --------
    token = st.session_state.get('jwt_token')
    payload = verify_token(token)

    if not payload:
        st.warning("Session expired. Please login again.")
        st.session_state['jwt_token'] = None
        st.session_state['page'] = "login"
        st.rerun()
        return

    email = payload.get("sub")
    ADMIN_EMAIL = os.getenv("ADMIN_EMAIL_ID")

    # -------- ADMIN ACCESS CONTROL --------
    if email != ADMIN_EMAIL:
        st.error("Access Denied. Admins only.")
        return

    # -------- PAGE HEADER --------
    st.title("🛡️ Admin Panel")
    st.markdown("Manage registered users below.")

    st.markdown("---")

    # -------- FETCH USERS --------
    users = db.get_all_users()

    if not users:
        st.info("No users found.")
        return

    # -------- TABLE HEADER --------
    header1, header2, header3 = st.columns([3, 2, 1])
    header1.markdown("**Email**")
    header2.markdown("**Created At**")
    header3.markdown("**Action**")

    st.markdown("---")

    # -------- USER LIST --------
    for user_email, created_at in users:

        col1, col2, col3 = st.columns([3, 2, 1])

        col1.write(user_email)
        col2.write(created_at)

        # Prevent deleting admin
        if user_email != ADMIN_EMAIL:
            if col3.button("Delete", key=f"del_{user_email}"):

                db.delete_user(user_email)

                st.warning(f"{user_email} deleted.")
                time.sleep(1)
                st.rerun()
        else:
            col3.write("Protected")

    st.markdown("---")

    # -------- STATS SECTION --------
    st.subheader("📊 System Stats")

    total_users = len(users)

    s1, s2 = st.columns(2)
    s1.metric("Total Users", total_users)
    s2.metric("Admin Email", ADMIN_EMAIL)

if st.session_state['jwt_token']:

    # -------- SIDEBAR NAV --------
    with st.sidebar:
        st.title("🤖 Policy Nav")

        payload = verify_token(st.session_state['jwt_token'])
        if not payload:
          st.session_state['jwt_token'] = None
          st.session_state['page'] = "login"
          st.rerun()
        email = payload.get("sub")
        ADMIN_EMAIL = os.getenv("ADMIN_EMAIL_ID")


        menu_options = ["Dashboard", "Readability"]

        if email == ADMIN_EMAIL:
            menu_options.append("Admin")

        selected = option_menu(
            menu_title="Navigation",
            options=menu_options,
            icons=["house", "book", "shield"][:len(menu_options)],
            default_index=0
        )

        st.markdown("---")

        if st.button("Logout", use_container_width=True):
            st.session_state['jwt_token'] = None
            st.session_state['page'] = "login"
            st.rerun()

    # -------- MAIN AREA --------
    if selected == "Dashboard":
        dashboard_page()
    elif selected == "Readability":
        readability_page()
    elif selected == "Admin":
        admin_page()

else:
    if st.session_state['page'] == 'login':
        login_page()
    elif st.session_state['page'] == 'signup':
        signup_page()
    elif st.session_state['page'] == 'forgot':
        forgot_password_page()





Overwriting app.py


In [14]:
from pyngrok import ngrok
import subprocess
import time
import os
import socket

print("Starting app...")

# Stop old Streamlit
os.system("pkill -f streamlit")

# Stop old ngrok tunnels
ngrok.kill()

# Get ngrok token
NGROK_TOKEN = os.getenv("NGROK_AUTHTOKEN")

if not NGROK_TOKEN:
    raise ValueError("NGROK_AUTHTOKEN not found.")

ngrok.set_auth_token(NGROK_TOKEN)

# Start Streamlit
subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501"]
)

# Wait until Streamlit is ready
def wait_for_streamlit(port=8501, timeout=20):
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection(("localhost", port), timeout=1):
                return True
        except OSError:
            time.sleep(0.5)
    return False

if not wait_for_streamlit():
    raise RuntimeError("Streamlit failed to start.")

# Create public link
public_url = ngrok.connect(8501).public_url

print("App live at:")
print(public_url)

Starting app...
App live at:
https://unparadoxal-constrictedly-lurline.ngrok-free.dev
